In [1]:
import numpy as np
from scipy.constants import c
from tqdm.notebook import tqdm
import h5py as hp
import matplotlib.pyplot as plt
from aps_figures.aps_one_column import *
from FITX import fit_risetime
from mbtrack2 import Synchrotron, Electron
from mbtrack2.utilities import Optics
from mbtrack2.tracking import LongitudinalMap, SynchrotronRadiation, TransverseMap
from mbtrack2.tracking import Beam, Bunch, WakePotential
from mbtrack2.tracking import RFCavity, SynchrotronRadiation
from mbtrack2.tracking.monitors import BunchMonitor, WakePotentialMonitor, BeamMonitor
from mbtrack2 import LongRangeResistiveWall
particle = Electron()
from machine_data.TDR2 import *

In [2]:
LongRangeResistiveWall?

Init signature:
LongRangeResistiveWall(
    ring,
    beam,
    length,
    rho,
    radius,
    types=['Wlong', 'Wxdip', 'Wydip'],
    nt=50,
    x3=None,
    y3=None,
)
Docstring:     
Element to deal with multi-bunch and multi-turn wakes from resistive wall 
using the algorithm defined in [1].

Main approximations:
    - Bunches are treated as point charge.
    - Assymptotic expression for the resistive wall wake functions are 
    used.
    - Multi-turn wakes are limited to nt turns.

Self-bunch interaction is not included in this element and should be dealed
with the WakePotential class.

Parameters
----------
ring : Synchrotron object
beam : Beam object
length : float
    Length of the resistive pipe to consider in [m].
rho : float
    Effective resistivity to consider in [ohm.m] as in [1].
radius : float
    Beam pipe radius to consider in [m].
types : str or list, optional
    Wake types to consider. 
    The default is ["Wlong","Wxdip","Wydip"].
nt : int or float, optional
   

In [3]:
from SOLEILII_parameters.SOLEILII_TDR_parameters import *
chro = [1.6, 1.6] # Horizontal and vertical (non-normalized) chromaticities.
turns=20000

In [4]:
from machine_data.soleil import v2366
ring2 = v2366(IDs='close', load_lattice=False)
optics = ring2.optics

In [5]:
ring = Synchrotron(h=ring2.h, optics=optics, particle=particle, L=ring2.L, E0=ring2.E0, ac=ring2.ac, 
                   U0=ring2.U0, tau=ring2.tau, emit=ring2.emit, tune=ring2.tune, 
                   sigma_delta=ring2.sigma_delta, sigma_0=ring2.sigma_0, chro=chro)
print(ring.optics.beta(0))

[[3.288]
 [4.003]]


In [6]:
per_bunch_current = 1.2e-3
filling_pattern = np.ones(ring.h)*per_bunch_current
mybeam = Beam(ring)
mybeam.init_beam(filling_pattern, mp_per_bunch=1)
bunch_monitor = BunchMonitor(0, save_every=1, buffer_size=500, total_size=turns, file_name='test_monitor', mpi_mode=False)
beam_monitor = BeamMonitor(ring.h, save_every=1, buffer_size=500, total_size=turns, mpi_mode=False)

In [7]:
long_map = LongitudinalMap(ring)
print('RF cavity voltage phase {:.2f}'.format(np.arccos(ring.U0/V_RF)))
rf = RFCavity(ring, m=1, Vc=V_RF, theta=np.arccos(ring.U0/V_RF))
sr = SynchrotronRadiation(ring, switch=[1, 0, 0])
trans_map = TransverseMap(ring)
wakemodel = load_TDR2_wf(version='TDR2.1_IDopen')
wakemodel.drop(['Wlong', 'Zlong', 'Zxdip', 'Zydip', 'Wxdip'])
short_wakefield = WakePotential(ring, wakemodel, n_bin=65)
#beta_y for LongRangeResistiveWall 3.99 m
long_wakefield = LongRangeResistiveWall(
    ring=ring,
    beam=mybeam,
    length=350.749,
    rho=2.135e-8,
    radius=5e-3,
    types=['Wydip'],
    nt=1,
    x3=6.38e-3,
    y3=6.73e-3,
)
# wakepotential_monitor = WakePotentialMonitor(bunch_number=0, wake_types='Wydip', n_bin=100, save_every=1, 
                 # buffer_size=500, total_size=turns, file_name=None, mpi_mode=False)

RF cavity voltage phase 1.32


In [8]:
for i in tqdm(range(turns)):
    trans_map.track(mybeam)
    long_map.track(mybeam)
    rf.track(mybeam)
    # short_wakefield.track(mybeam)
    long_wakefield.track(mybeam)
    sr.track(mybeam)
    # wakepotential_monitor.track(mybeam, wakefield)
    beam_monitor.track(mybeam)
    bunch_monitor.track(mybeam)

  0%|          | 0/20000 [00:00<?, ?it/s]